In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
# !pip install esy-osm-pbf
# !pip install -U osmium
!pip install hdbscan

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

import esy.osm.pbf
import osmium
import xml

import hdbscan
import sklearn.manifold


from util import get_data
from util import mapping
from util import clean_data
from util import config

In [3]:
rides = clean_data.load_ridewgps_rides()
clean_data.clean_ridewgps_df(rides)
rides.head()

/home/emily/anaconda3/envs/insight/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,id,distance,elevation_gain,description,name,avg_speed,user_id,update_days,if_updated,elevation_net,photos,big_user,crow_distance
749,23209396,2.659848,37.6219,,04/30/18,5.002846,50396,0,False,5.2647,False,True,1.116021e-04
790,54372107,2.762263,43.3334,,08/15/20,5.171163,50396,0,False,-9.2157,False,True,2.040744e-08
797,48496079,2.777424,146.6010,,05/08/20,5.518063,490190,0,False,26.6880,False,False,6.240184e-06
798,40433541,2.779512,17.4177,,09/23/19,5.192650,268854,0,False,4.6607,False,True,8.141466e-10
805,47936294,2.795935,91.2606,,04/28/20,5.470311,490190,0,False,-2.3147,False,False,4.518103e-09


In [7]:
rides.avg_speed.mean()

13.70010836680067

In [ ]:
useful_cols = ['distance', 'elevation_gain', 'avg_speed',
               'if_updated', 'photos', 'big_user', 'crow_distance']
rides_use = rides[useful_cols].copy()
rides_use.dtypes
for col in rides_use.columns:
    if rides_use[col].dtype == 'float64':
        rides_use[col] = rides_use[col] / rides_use[col].max()


In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=100)
clusterer.fit(rides_use)
print(clusterer.labels_.max())

In [ ]:
mapped_clusters = sklearn.manifold.TSNE(n_components=2, metric='cosine', init='pca').fit_transform(rides[useful_cols])


In [ ]:
x = mapped_clusters[:, 0]
y = mapped_clusters[:, 1]
# colours = ['firebrick', 'tomato', 'darkorange', 'burlywood',
#           'gold', 'olivedrab', 'darkseagreen', 'darkgreen',
#           'lightseagreen', 'paleturquoise', 'darkslategray',
#           'deepskyblue','steelblue', 'royalblue', 'slateblue',
#           'darkorchid', 'fuchsia', 'palevioletred']
colours = sns.color_palette('deep', 10)
plt.figure(figsize=(10, 4))
for c in range(10):
    plt.plot(x[clusterer.labels_ == c], y[clusterer.labels_ == c], '.', color=colours[c])

In [ ]:
clusterer.condensed_tree_.plot(select_clusters=True,
                              selection_palette=colours)

In [ ]:
clusterer.single_linkage_tree_.plot()

In [ ]:
for col in rides.columns:
    print('{}: {} missing'.format(col, rides[col].isna().sum()))

In [ ]:
a = rides.group_membership_id.value_counts()
a[a > 100].index.tolist()

In [ ]:
# rides[rides.if_update > 0].shape
22670 - 21788

In [ ]:
trips = pd.read_csv(config.RAW_DATA_PATH + 'ridewgps_trips.csv')
trips = clean_data.clean_trips(trips)
trips.shape
# routes = pd.read_csv(config.RAW_DATA_PATH + 'ridewgps_routes.csv')
# trip_cols = list(set(trips.columns) - set(routes.columns))

In [ ]:
for col in rides.columns:
    print('{} ({} missing):\n{}'.format(col, rides[col].isna().sum(), rides[col].value_counts()))

In [ ]:
# trips[trips.is_gps == False][['id', 'moving_time', 'duration']]
a = rides.user_id.value_counts()
a[(a > 500)]

In [ ]:
rides.highlighted_photo_id[rides.highlighted_photo_id > 0].shape

In [ ]:
osm = esy.osm.pbf.File('/media/sf_Insight/data/north-america-latest.osm.pbf')
for i, entry in enumerate(osm):
    if i == 2: break
entry

In [ ]:

class CounterHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0

    def node(self, n):
        self.num_nodes += 1
        
if __name__ == '__main__':

    h = CounterHandler()
    filename = '/home/emily/Downloads/map_ashokan.xml'
    h.apply_file(filename)

    print("Number of nodes: %d" % h.num_nodes)

In [ ]:
class LatLonHandler(osmium.SimpleHandler):
    def __init__(self):
        super(LatLonHandler).__init__()
        self.latlons = []

    def node(self, n):
        self.latlons.append([o.tags['place']])
        self.num_nodes += 1

filename = '/home/emily/Downloads/map_ashokan.xml'
h = LatLonHandler()
h.apply_file(filename)

In [ ]:
root = tree.getroot()
root[3].text

In [ ]:
node[1].